<br>
@Author: Ayush Prajapati<br>
@Date: 11-09-2024<br>
@Last Modified by: Ayush Prajapati<br>
@Last Modified time: 11-09-2024<br>
@Title: S3 CRUD operations using Boto3 Library<br>
<br>

In [1]:
%pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [9]:
import boto3
from dotenv import load_dotenv
from tqdm import tqdm
from boto3.s3.transfer import TransferConfig
import os

In [3]:
load_dotenv()

True

**Create a boto3 client**

In [4]:
s3 = boto3.client('s3')

In [5]:
response = s3.list_buckets()

# Output bucket names
for bucket in response['Buckets']:
    print(bucket['Name'])

ayush-boto-crud-bucket
ayush-demo-bucket1
ayushp-static-website


### Uploading large file

In [6]:
# Setup multipart transfer configuration 
config = TransferConfig(
    multipart_threshold=1024 * 1024 * 8,  # 8 MB threshold for multipart upload
    max_concurrency=10,                    # 10 concurrent threads
    multipart_chunksize=1024 * 1024 * 8,   # 8 MB chunk size
    use_threads=True
)

In [7]:
def upload_large_file_with_progress(local_file_name, bucket_name, s3_key):
    """
    Description:
        Uploads a large file to the S3 bucket
    Parameters:
        local_file_name: name of local file
        bucket_name = name of bucket
        s3_key = key value in s3
    Returns:
        None
    """
    # Get file size for the progress bar
    file_size = os.path.getsize(local_file_name)

    with tqdm(total=file_size, unit='B', unit_scale=True, desc=local_file_name) as pbar:
        def progress_hook(bytes_transferred):
            pbar.update(bytes_transferred)

        s3.upload_file(local_file_name, bucket_name, s3_key, Config=config, Callback=progress_hook)

In [10]:
def main():
    bucket_name = 'ayush-demo-bucket1'
    file_name = '/home/ayush/Videos/Immaculate-2024.mkv'
    s3_key = 'Immaculate-2024.mkv'

    upload_large_file_with_progress(file_name, bucket_name, s3_key)

    
if __name__ == '__main__':
    main()

/home/ayush/Videos/Immaculate-2024.mkv:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

/home/ayush/Videos/Immaculate-2024.mkv: 100%|██████████| 1.54G/1.54G [13:02<00:00, 1.97MB/s]  


### Downloading large file

In [21]:
# Setup multipart transfer configuration 
config = TransferConfig(
    multipart_threshold=1024 * 1024 * 8,  # 8 MB threshold for multipart upload
    max_concurrency=10,                    # 10 concurrent threads
    multipart_chunksize=1024 * 1024 * 8,   # 8 MB chunk size
    use_threads=True
)

In [19]:
def download_large_file_from_s3(bucket_name, s3_key, file_name):
    """
    Description:
        Downloads a large file from the S3 bucket
    Parameters:
        bucket_name = name of bucket
        s3_key = key value in s3
        file_name: name of file to be stored
    Returns:
        None
    """
    try:
        # Get the file size for the progress bar
        file_size = s3.head_object(Bucket=bucket_name, Key=s3_key)['ContentLength']

        # Create a progress bar
        with tqdm(total=file_size, unit='B', unit_scale=True, desc=s3_key) as pbar:
            with open(file_name, 'wb') as f:
                # Define a callback function to update the progress bar
                def progress_hook(bytes_transferred):
                    pbar.update(bytes_transferred)

                # Download the file in chunks
                s3.download_fileobj(bucket_name, s3_key, f, Config = config, Callback=progress_hook)

        print(f"File {s3_key} downloaded successfully from {bucket_name} to {file_name}")
    except Exception as e:
        print(f"Error downloading file: {e}")

In [23]:
def main():
    bucket_name = 'ayush-demo-bucket1'
    file_name = '/home/ayush/Videos/AWS-Immaculate-2024.mkv'
    s3_key = 'Immaculate-2024.mkv'

    download_large_file_from_s3(bucket_name, s3_key, file_name)


if __name__ == '__main__':
    main()

Immaculate-2024.mkv: 100%|██████████| 1.54G/1.54G [01:25<00:00, 18.1MB/s]

File Immaculate-2024.mkv downloaded successfully from ayush-demo-bucket1 to /home/ayush/Videos/AWS-Immaculate-2024.mkv
